# Importando

In [1]:
import matplotlib.pyplot as plt
import pandas as pd   
import warnings

warnings.filterwarnings('ignore')

def ColabOuCode(path_drive, path_local, ncolunas=None):
    '''
    Importa o DataFrame de acordo com a plataforma usada. Retorna um DataFrame.
    '''
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        df = pd.read_csv(path_drive)
    except:
        if ncolunas is None: df = pd.read_csv(path_local)
        else: df = pd.read_csv(path_local,  nrows=ncolunas)
    return df


In [2]:
drive = '/content/drive/MyDrive/Colab Notebooks/TERA-desafio-3/data/dados_tratados.csv'
local = './data/dados_tratados.csv'

df = ColabOuCode(
    drive,
    local,
    ncolunas=None
)

df.head()


,Unnamed: 0,oldbalanceOrig,origReinc,oldbalanceDest,destReinc,amount,type_CASH_OUT,type_TRANSFER,isFraud
0,0,5.198497,0,0.000000,0,5.198497,0,1,0
1,1,5.198497,0,9.960907,0,5.198497,1,0,0
2,2,9.637241,0,8.533657,0,12.342062,1,0,1
3,3,6.558198,0,10.017932,0,12.279836,0,1,1
4,4,9.290537,0,8.743053,0,12.649751,0,1,1


In [4]:
df.columns

Index(['Unnamed: 0', 'oldbalanceOrig', 'origReinc', 'oldbalanceDest',
       'destReinc', 'amount', 'type_CASH_OUT', 'type_TRANSFER', 'isFraud'],
      dtype='object')

In [ ]:
df.info()

# Classificações

In [ ]:
from sklearn.model_selection import train_test_split

x_df = df.drop('isFraud',axis=1)
y_df = df[['isFraud']]

X_treino, X_teste, y_treino, y_teste = train_test_split(x_df, y_df, test_size=0.8, random_state=42)


In [ ]:
## Regressão Logística

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix


regLogistica = LogisticRegression(
    max_iter=100,
    class_weight='balanced',
)

regLogistica.fit(X_treino, y_treino)

y_regPredicao = regLogistica.predict(X_teste)

confusion_matrix(y_teste, y_regPredicao)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_teste, y_regPredicao, target_names=['Fraude', 'Nao Fraude']))

## Árvore de Decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

arvere = DecisionTreeClassifier(
    criterion='gini',
    class_weight='balanced',

)
arvere.fit(X_treino, y_treino)

y_arvPred = arvere.predict(X_teste)

#accuracy_score(y_teste, y_pred, normalize=True)

matrizConf = confusion_matrix(y_teste, y_arvPred)
matrizConf

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

matrizConf = confusion_matrix(y_teste, y_pred)
matrizDisplay = ConfusionMatrixDisplay(confusion_matrix=matrizConf, display_labels=arvere.classes_)
matrizDisplay.plot(values_format='d')


In [ ]:
from sklearn.metrics import precision_score, recall_score, fbeta_score


print('Precision ',precision_score(y_teste, y_pred, pos_label=0))
print('Recall ', recall_score(y_teste, y_pred, pos_label=0))
print('F1', fbeta_score(y_teste, y_pred, beta=1, pos_label=0))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_teste, y_pred, target_names=['Fraude', 'Nao Fraude']))

In [ ]:
'''
export_graphviz(
    arvere,
    out_file=('arvere.dot'),
    feature_names=X_treino.columns,
    rounded=True,
    filled=True
)

!dot -Tpng arvere.dot -o iris_tree.png
'''

## XGBoost